### Imports

#### Import Packages

In [1]:
import polars as pl
import altair as alt
import numpy as np
from great_tables import GT
import gc

# https://altair-viz.github.io/user_guide/display_frontends.html#displaying-in-jupyterlab
alt.renderers.enable('mimetype')
# alt.JupyterChart.enable_offline()
# alt.renderers.enable("jupyter", offline=True)

RendererRegistry.enable('mimetype')

#### Import Panel Data

In [2]:
grocery_lf = pl.scan_csv(source="data/panel-datasets/edible_grocery.csv",
                         has_header=True,
                         separator=",",
                         schema={'panel_id': pl.Int32,
                                 'trans_id': pl.Int32,
                                 'week': pl.Int16,
                                 'sku_id': pl.Categorical,
                                 'units': pl.Int16,
                                 'price': pl.Float32,
                                 'brand': pl.Categorical})
grocery_lf.head().collect()

panel_id,trans_id,week,sku_id,units,price,brand
i32,i32,i16,cat,i16,f32,cat
3102011,1569,6,"""19""",1,2.79,"""Alpha"""
3102012,4301,15,"""15""",1,3.19,"""Alpha"""
3102012,4301,15,"""15""",1,3.19,"""Alpha"""
3102012,4301,15,"""38""",1,3.49,"""Bravo"""
3102012,4301,15,"""44""",1,3.49,"""Bravo"""


In [3]:
sku_lf = pl.scan_csv(source="data/panel-datasets/sku_weight.csv",
                         has_header=True,
                         separator=",",
                         schema={'sku_id': pl.Categorical,
                                 'weight': pl.Int16})
sku_lf.head().collect()

sku_id,weight
cat,i16
"""1""",400
"""2""",400
"""3""",400
"""4""",250
"""5""",1000


In [4]:
kiwi_lf = pl.scan_csv(source="data/panel-datasets/kiwibubbles_trans.csv",
                      has_header=True,
                      separator=",",
                      schema={'ID': pl.Int16,
                              'Market': pl.Categorical,
                              'Week': pl.Int16,
                              'Day': pl.Int16,
                              'Units': pl.Int16})
kiwi_lf.head().collect()

ID,Market,Week,Day,Units
i16,cat,i16,i16,i16
10001,"""1""",19,3,1
10002,"""1""",12,5,1
10003,"""1""",37,7,1
10004,"""1""",30,6,1
10004,"""1""",47,3,1


#### Reusable Functions

In [5]:
# Weekly Grocery Sales LazyFrame (Query Plan): Weekly 'spend' by 'Category', 'Brand' or 'All'
def weekly_spend_summary(brand, lf):
    summary = (
        lf
        .select(['week', 'units', 'price', 'brand'])
        .with_columns(((pl.col('units') * pl.col('price'))).alias('spend'))
    )
    
    if brand == 'Category': # Return LazyFrame of total category
        summary = summary.group_by('week')
    elif brand == 'All': # Return LazyFrame of all brands
        summary = summary.group_by('week', 'brand')
    else:  # Return LazyFrame of specified brand
        summary = summary.filter(
            pl.col('brand') == brand
        ).group_by('week', 'brand')
        
    summary = summary.agg(
        pl.col("spend").sum().alias('Weekly Spend') 
    ).sort('week')
    
    return summary

In [6]:
# Weekly Grocery Volume Sales LazyFrame (Query Plan): Weekly 'volume' by 'Brand' or 'All'
def weekly_vol_summary(brand, lf):
    with pl.StringCache():
        lf = (
            lf
            .join(
                other=sku_lf,
                left_on="sku_id",
                right_on="sku_id"            
            )
            .select(['week', 'units', 'brand', 'weight'])
        )
        
        if brand != 'All': 
            brand = [brand] if type(brand) == str else brand
            lf = lf.filter(
                pl.col('brand').is_in(*[brand])
            )
            
        summary = lf.with_columns(
            (((pl.col('units') * pl.col('weight'))/1000)).alias('volume')
        ).group_by('week', 'brand').agg(
            pl.col("volume").sum().alias('Weekly Volume')
        ).sort('week')
        
    return summary

In [7]:
# Altair Weekly Line Plot
def weekly_plot(dataframe, y, color=None, title="", y_axis_label="", pct=False, legend=False):
    
    # Configure the color encoding only if color is provided
    if color is not None:
        color_encoding = alt.Color(
            f'{color}:N',  # N = a discrete unordered category
            legend=alt.Legend(title=color) if legend else None  # Add legend conditionally
        )
    else:
        color_encoding = alt.Color()  # No color encoding    
    
    chart = alt.Chart(dataframe).mark_line(strokeWidth=1).encode(
        x = alt.X(
            'week',
            axis=alt.Axis(
                values=np.arange(0, 104 + 1, 13), # Explicitly specify quarter-end weeks
                labelExpr="datum.value", # Show only these labels
                title='Week'
            )
        ),
        y = alt.Y(
            f'{y}:Q', # Q = a continuous real-valued quantity
            title=y_axis_label,
            axis=alt.Axis(format="$,.0f") if not pct else alt.Axis(format=",.0%")
        ),
        color = color_encoding
    ).properties(
        width=650,
        height=250,
        title=title
    ).configure_view(
        stroke=None
    ).configure_axisY(
        # grid=False # turn off y-axis grid if required
    )

    return chart # alt.JupyterChart(chart)

In [8]:
# Annual Sales Summary LazyFrame for All Brands
def annual_sales_summary():
    summary = (
        weekly_spend_summary('All', grocery_lf)
        .with_columns((pl.col("week") / 52).ceil().alias('year'))
        .group_by(['year', 'brand'])
        .agg(pl.col("Weekly Spend").sum().alias('Yearly Sales'))
    ).sort('year')
    
    return summary

In [9]:
def freq_dist_plot(
    data, 
    column, 
    bin_edges, 
    labels, 
    x_title, 
    y_title, 
    chart_title, 
    subtitle, 
    width=650, 
    height=250, 
    label_angle=0, 
    left_closed=True, 
    compute_rel_freq=True
):
    """
    Creates a standardized Altair bar chart for relative frequency distribution plots.

    Parameters:
    - data (Polars LazyFrame or DataFrame): Input dataset.
    - column (str): Column to analyze for distribution.
    - bin_edges (array-like): Edges for binning.
    - labels (list of str): Labels for the bins.
    - x_title (str): Title for the x-axis.
    - y_title (str): Title for the y-axis.
    - chart_title (str): Main title for the chart.
    - subtitle (str): Subtitle for the chart.
    - width (int, optional): Width of the chart. Default is 650.
    - height (int, optional): Height of the chart. Default is 250.
    - label_angle (int, optional): Angle for x-axis labels. Default is 0.
    - left_closed (bool, optional): Whether bins are left-closed. Default is True.
    - compute_rel_freq (bool, optional): Whether to compute relative frequencies. Default is True.

    Returns:
    - alt.Chart: The generated Altair chart.
    """
    # Apply binning to the data
    binned_data = data.with_columns(
        pl.col(column).cut(bin_edges, labels=labels, left_closed=left_closed).alias("cut")
    )

    # Optionally compute relative frequencies
    if compute_rel_freq:
        binned_data = (
            binned_data
            .group_by("cut")
            .agg(pl.col("cut").count().alias("Frequency"))
            .with_columns(
                (pl.col("Frequency") / pl.col("Frequency").sum()).alias("% of Total")
            )
            .collect()
        )

    # Create the Altair chart
    chart = alt.Chart(binned_data).mark_bar().encode(
        x=alt.X("cut:O", axis=alt.Axis(labelAngle=label_angle, title=x_title), sort=labels),
        y=alt.Y("% of Total:Q", axis=alt.Axis(format=".0%", title=y_title)),
    ).properties(
        width=width,
        height=height,
        title={"text": chart_title, "subtitle": subtitle},
    )

    return chart

### Preliminaries

#### Weekly Sales Pattern

In [ ]:
# Weekly Sales Pivot Table - Polars DataFrame
# For visualizing and inspecting only
weekly_spend_summary('All', grocery_lf).collect().pivot(
    on="brand",
    index="week",
    values="Weekly Spend",
    sort_columns=True,
).with_columns(
    pl.sum_horizontal(pl.exclude('week')).alias("Total") # Row total
).sort("week")

week,Alpha,Bravo,Charlie,Delta,Other,Total
i16,f32,f32,f32,f32,f32,f64
1,331.460114,247.780121,51.790005,17.639999,18.75,667.420227
2,567.249939,398.640137,45.429996,34.190002,23.23,1068.740112
3,497.96994,639.980103,51.110001,39.389999,14.550001,1243.000122
4,1389.95752,472.5,51.869999,85.039993,39.779995,2039.147461
5,358.240204,252.540115,40.209995,69.540001,45.929993,766.460327
…,…,…,…,…,…,…
100,692.479919,420.479706,69.599991,54.590004,47.969997,1285.119629
101,265.770081,264.660065,64.029999,35.049999,26.880001,656.390198
102,379.610168,456.789429,43.499996,20.4,43.609993,943.909607


In [11]:
weekly_plot(dataframe=weekly_spend_summary('Category', grocery_lf).collect(), 
            y='Weekly Spend', 
            title='Category - Weekly Revenue', 
            y_axis_label='Spend ($)',
            pct=False,
            legend=False)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [12]:
weekly_plot(dataframe=weekly_spend_summary('Alpha', grocery_lf).collect(), 
            y='Weekly Spend', 
            color='brand',
            title='Alpha - Weekly Revenue', 
            y_axis_label='Spend ($)',
            pct=False,
            legend=False)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [13]:
weekly_plot(dataframe=weekly_spend_summary('Bravo', grocery_lf).collect(), 
            y='Weekly Spend', 
            color='brand',
            title='Bravo - Weekly Revenue', 
            y_axis_label='Spend ($)',
            pct=False,
            legend=False)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


#### Weekly (Volume) Market Share

In [14]:
# Weekly (volume) market share
pct_volume =(
    weekly_vol_summary('All', grocery_lf)
    .collect()
    .pivot(
        on='brand',
        index='week',
        values='Weekly Volume',
        sort_columns=True
    )
    .with_columns(
        (pl.col("*").exclude("week")) / pl.sum_horizontal(pl.exclude('week'))
    )
)
pct_volume

week,Alpha,Bravo,Charlie,Delta,Other
i16,f64,f64,f64,f64,f64
1,0.481077,0.355011,0.11194,0.02532,0.026652
2,0.524953,0.371525,0.059391,0.029696,0.014435
3,0.389338,0.508812,0.060676,0.032505,0.008668
4,0.677032,0.230511,0.036686,0.044984,0.010787
5,0.428735,0.324306,0.07574,0.119348,0.051871
…,…,…,…,…,…
100,0.545707,0.31711,0.080381,0.039087,0.017715
101,0.36891,0.40552,0.140805,0.059138,0.025627
102,0.342581,0.543494,0.071147,0.019689,0.023089


In [15]:
pct_volume_plot = (
    weekly_vol_summary('All', grocery_lf)
    .group_by('week')
    .agg(
        pl.col('Weekly Volume').sum().alias('Total Weekly Volume')
    )
    .join(
        other=weekly_vol_summary(['Alpha', 'Bravo'], grocery_lf),
        on='week',
    )
    .filter(
        (pl.col('brand').is_in(['Alpha', 'Bravo']))
    )
    .with_columns(
        # compute brand wise % of total volume sale
        (pl.col('Weekly Volume') / pl.col('Total Weekly Volume')).alias('pct_volume')
    )
).collect()

weekly_plot(dataframe=pct_volume_plot, 
            y='pct_volume', 
            color='brand', 
            title="", 
            y_axis_label="", 
            pct=True, 
            legend=True)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [16]:
corr_matrix = (
    pct_volume.select(
        pl.col('*').exclude('week')  # Exclude 'week' column
    )
    .corr()  # Compute the correlation matrix
    .with_columns(
        pl.Series(pct_volume.columns[1:]).alias("index")
    )
)

(
    GT(corr_matrix, rowname_col='index')
    .tab_header(title="Correlation Matrix of Weekly (Volume) Market Share")
    .fmt_number(columns=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'])
    .data_color(
        domain=[-1, 1],
        palette=["rebeccapurple", "white", 'orange']
    )
)

GT(_tbl_data=shape: (5, 6)
┌───────────┬───────────┬───────────┬───────────┬───────────┬─────────┐
│ Alpha     ┆ Bravo     ┆ Charlie   ┆ Delta     ┆ Other     ┆ index   │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---     │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆ f64       ┆ str     │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═════════╡
│ 1.0       ┆ -0.853061 ┆ -0.34158  ┆ -0.109693 ┆ -0.148193 ┆ Alpha   │
│ -0.853061 ┆ 1.0       ┆ -0.114179 ┆ -0.231641 ┆ -0.104983 ┆ Bravo   │
│ -0.34158  ┆ -0.114179 ┆ 1.0       ┆ 0.131988  ┆ 0.243434  ┆ Charlie │
│ -0.109693 ┆ -0.231641 ┆ 0.131988  ┆ 1.0       ┆ 0.308354  ┆ Delta   │
│ -0.148193 ┆ -0.104983 ┆ 0.243434  ┆ 0.308354  ┆ 1.0       ┆ Other   │
└───────────┴───────────┴───────────┴───────────┴───────────┴─────────┘, _body=<great_tables._gt_data.Body object at 0x10da797f0>, _boxhead=Boxhead([ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10da792b0>, _spanners=Spanners([]), _heading=Heading(title='Correlation Matrix of Weekly (Volume) Market Share', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ffa500')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7c51a8')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[2]), grpname=None, colname='Alpha', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#cbb9dc')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[3]), grpname=None, colname='Alpha', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#eee9f4')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[4]), grpname=None, colname='Alpha', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e8e1f0')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7c51a8')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=Non

#### Annual Sales

In [17]:
annual_sales_pivot = annual_sales_summary().collect().pivot(
    on='brand',
    index='year',
    sort_columns=True
).with_columns(
    pl.sum_horizontal(pl.all().exclude('year')).alias("Total") # add totals column
)

(
    GT(annual_sales_pivot, rowname_col="year")
    .tab_header(title="Annual Sales ($) by Brand")
    .tab_stubhead(label="Year")
    .fmt_currency()
    .data_color(
        columns=annual_sales_pivot.columns[:-1],
        domain=[100, 36_000],
        palette=["white", "rebeccapurple"]
    )    
)

GT(_tbl_data=shape: (2, 7)
┌──────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┬──────────────┐
│ year ┆ Alpha        ┆ Bravo        ┆ Charlie     ┆ Delta       ┆ Other       ┆ Total        │
│ ---  ┆ ---          ┆ ---          ┆ ---         ┆ ---         ┆ ---         ┆ ---          │
│ f64  ┆ f32          ┆ f32          ┆ f32         ┆ f32         ┆ f32         ┆ f32          │
╞══════╪══════════════╪══════════════╪═════════════╪═════════════╪═════════════╪══════════════╡
│ 1.0  ┆ 33570.949219 ┆ 28603.345703 ┆ 5120.869629 ┆ 3271.509766 ┆ 1535.22998  ┆ 72101.90625  │
│ 2.0  ┆ 35250.738281 ┆ 26926.847656 ┆ 3922.679688 ┆ 2820.810303 ┆ 1739.820068 ┆ 70660.898438 │
└──────┴──────────────┴──────────────┴─────────────┴─────────────┴─────────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x10db149d0>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.stub: 2>, column_label='year', column_align='right', column_width=None), ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Total', type=<ColInfoTypeEnum.default: 1>, column_label='Total', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10da91310>, _spanners=Spanners([]), _heading=Heading(title='Annual Sales ($) by Brand', subtitle=None, preheader=None), _stubhead='Year', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='year', rows=[0]), grpname=None, colname='year', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[1]), grpname=None, colname='year', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7041a0')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#69379b')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#865dae')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8d67b3')]), StyleInfo(locname=LocBody(columns='Charlie', rows=[0]), grpname=None, colname='Charlie', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, s

In [18]:
annual_change_sales = annual_sales_pivot.with_columns(
    pl.col('*').exclude('year').pct_change()
).filter(
    pl.col('year') == 2
).unpivot(
    index="year",  # Keep 'year' as a fixed identifier
    value_name='% Change',  
    variable_name='brand'
)

(
    GT(annual_change_sales, rowname_col='brand')
    .tab_header(title='% Change in Annual Sales')
    .tab_stubhead(label="Brands")
    .fmt_percent()
    .data_color(
        domain=[-0.3, 0.3],
        palette=['orange', 'white', 'rebeccapurple']
    ) 
    .cols_hide('year')   
    .cols_label(brand='Y-o-Y % Change')
)

GT(_tbl_data=shape: (6, 3)
┌──────┬─────────┬───────────┐
│ year ┆ brand   ┆ % Change  │
│ ---  ┆ ---     ┆ ---       │
│ f64  ┆ str     ┆ f32       │
╞══════╪═════════╪═══════════╡
│ 2.0  ┆ Alpha   ┆ 0.050037  │
│ 2.0  ┆ Bravo   ┆ -0.058612 │
│ 2.0  ┆ Charlie ┆ -0.233982 │
│ 2.0  ┆ Delta   ┆ -0.137765 │
│ 2.0  ┆ Other   ┆ 0.133263  │
│ 2.0  ┆ Total   ┆ -0.019986 │
└──────┴─────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x10da958c0>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.hidden: 4>, column_label='year', column_align='right', column_width=None), ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='Y-o-Y % Change', column_align='left', column_width=None), ColInfo(var='% Change', type=<ColInfoTypeEnum.default: 1>, column_label='% Change', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10da93d90>, _spanners=Spanners([]), _heading=Heading(title='% Change in Annual Sales', subtitle=None, preheader=None), _stubhead='Brands', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='year', rows=[0]), grpname=None, colname='year', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[1]), grpname=None, colname='year', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[2]), grpname=None, colname='year', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[3]), grpname=None, colname='year', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[4]), grpname=None, colname='year', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[5]), grpname=None, colname='year', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', r

In [19]:
chart = alt.Chart(annual_sales_summary().collect()).mark_bar().encode(
    x=alt.X("brand:N", 
            axis=alt.Axis(labelAngle=0)),
    xOffset="year:N",
    y=alt.Y("Yearly Sales:Q",
            axis=alt.Axis(format="$,.0f")),
    color=alt.Color("year:N"),
).properties(
    width=650,
    height=250,
    title='Year 1 & Year 2 Revenues'
).show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [20]:
base = alt.Chart(annual_change_sales).encode(
    x=alt.X("brand:N", 
            axis=alt.Axis(labelAngle=0)
    ),
    y=alt.Y("% Change:Q",
            axis=alt.Axis(format=".0%")
    ),
    color=alt.condition(
        alt.datum["% Change"] > 0, # fixed this 
        alt.value("green"),  # The positive color
        alt.value("red")  # The negative color
    ),
    text=alt.Text(
        '% Change',
        format=(".1%")
    )
        
).properties(
    width=650,
    height=250,
    title='Y-o-Y % Change in Revenue'
)

chart = base.mark_bar() + base.mark_text(align='center', 
                                 baseline=alt.expr(alt.expr.if_(alt.datum['% Change'] >= 0, 'bottom', 'top')),
                                 dy=alt.expr(alt.expr.if_(alt.datum['% Change'] >= 0, -2, 2)),
                                 dx=0)
chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


#### Annual Market Share

In [21]:
market_share = annual_sales_pivot.with_columns(
    pl.col('*').exclude('year') / pl.col('Total')
)

(
    GT(market_share, rowname_col="year")
    .tab_header(title="Annual Market Share (%)")
    .tab_stubhead(label="Year")
    .fmt_percent()
    .cols_hide('Total')
    .data_color(
        domain=[0, 0.5],
        palette=["white", "rebeccapurple"]
    )    
)

GT(_tbl_data=shape: (2, 7)
┌──────┬──────────┬──────────┬──────────┬──────────┬──────────┬───────┐
│ year ┆ Alpha    ┆ Bravo    ┆ Charlie  ┆ Delta    ┆ Other    ┆ Total │
│ ---  ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---   │
│ f64  ┆ f32      ┆ f32      ┆ f32      ┆ f32      ┆ f32      ┆ f32   │
╞══════╪══════════╪══════════╪══════════╪══════════╪══════════╪═══════╡
│ 1.0  ┆ 0.465604 ┆ 0.396707 ┆ 0.071023 ┆ 0.045373 ┆ 0.021293 ┆ 1.0   │
│ 2.0  ┆ 0.498872 ┆ 0.381071 ┆ 0.055514 ┆ 0.03992  ┆ 0.024622 ┆ 1.0   │
└──────┴──────────┴──────────┴──────────┴──────────┴──────────┴───────┘, _body=<great_tables._gt_data.Body object at 0x10da5fc50>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.stub: 2>, column_label='year', column_align='right', column_width=None), ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Total', type=<ColInfoTypeEnum.hidden: 4>, column_label='Total', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10db14e90>, _spanners=Spanners([]), _heading=Heading(title='Annual Market Share (%)', subtitle=None, preheader=None), _stubhead='Year', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='year', rows=[0]), grpname=None, colname='year', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[1]), grpname=None, colname='year', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7141a0')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#865dae')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8a64b1')]), StyleInfo(locname=LocBody(columns='Charlie', rows=[0]), grpname=None, colname='Charlie', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e9e2f1')]), StyleInfo(locname=LocBody(columns='Ch

In [22]:
annual_change_share = market_share.drop('Total').with_columns(
    pl.col('*').exclude('year').pct_change()
).filter(
    pl.col('year') == 2
).unpivot(
    index="year",  # Keep 'year' as a fixed identifier
    value_name='% Change',  
    variable_name='brand'
)

(
    GT(annual_change_share, rowname_col='brand')
    .tab_header(title='% Change in Market Share')
    .tab_stubhead(label="Brands")
    .fmt_percent()
    .data_color(
        domain=[-0.3, 0.3],
        palette=['orange', 'white', 'rebeccapurple']
    ) 
    .cols_hide('year')   
    .cols_label(brand='Y-o-Y % Change')
)

GT(_tbl_data=shape: (5, 3)
┌──────┬─────────┬───────────┐
│ year ┆ brand   ┆ % Change  │
│ ---  ┆ ---     ┆ ---       │
│ f64  ┆ str     ┆ f32       │
╞══════╪═════════╪═══════════╡
│ 2.0  ┆ Alpha   ┆ 0.071451  │
│ 2.0  ┆ Bravo   ┆ -0.039414 │
│ 2.0  ┆ Charlie ┆ -0.21836  │
│ 2.0  ┆ Delta   ┆ -0.120181 │
│ 2.0  ┆ Other   ┆ 0.156374  │
└──────┴─────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x10db0fa10>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.hidden: 4>, column_label='year', column_align='right', column_width=None), ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='Y-o-Y % Change', column_align='left', column_width=None), ColInfo(var='% Change', type=<ColInfoTypeEnum.default: 1>, column_label='% Change', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10db150f0>, _spanners=Spanners([]), _heading=Heading(title='% Change in Market Share', subtitle=None, preheader=None), _stubhead='Brands', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='year', rows=[0]), grpname=None, colname='year', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[1]), grpname=None, colname='year', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[2]), grpname=None, colname='year', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[3]), grpname=None, colname='year', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[4]), grpname=None, colname='year', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum=None, styles=

In [23]:
base = alt.Chart(annual_change_share).encode(
    x=alt.X("brand:N", 
            axis=alt.Axis(labelAngle=0)
    ),
    y=alt.Y("% Change:Q",
            axis=alt.Axis(format=".0%")
    ),
    color=alt.condition(
        alt.datum["% Change"] > 0, # fixed this 
        alt.value("green"),  # The positive color
        alt.value("red")  # The negative color
    ),
    text=alt.Text(
        '% Change',
        format=(".1%")
    )
        
).properties(
    width=650,
    height=250,
    title='% Change in Annual Market Share'
)

chart = base.mark_bar() + base.mark_text(align='center', 
                                 baseline=alt.expr(alt.expr.if_(alt.datum['% Change'] >= 0, 'bottom', 'top')),
                                 dy=alt.expr(alt.expr.if_(alt.datum['% Change'] >= 0, -3, 3)),
                                 dx=0)

chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


#### Average SKU Price

In [24]:
avg_sku_price = grocery_lf.select(
    ['week', 'sku_id', 'price', 'brand']
).filter(
    (pl.col('week') <= 52) & # pricing in the first year
    (pl.col('brand') == 'Alpha') # Filter by Alpha brand 
).group_by('brand', 'sku_id').agg(
    pl.col('price').mean()
).drop('brand').sort(
    pl.col('sku_id').cast(pl.Int8)
)

(
    GT(avg_sku_price.collect(), rowname_col='sku_id') 
    .tab_header(title='Alpha - Average SKU Retail Prices')
    .tab_stubhead(label="SKU IDs")
    .fmt_currency()
    .data_color(
        domain=[1, 15],
        palette=['white', 'rebeccapurple']
    )  
    .cols_label(price='Average Price ($)')
)

GT(_tbl_data=shape: (18, 2)
┌────────┬──────────┐
│ sku_id ┆ price    │
│ ---    ┆ ---      │
│ cat    ┆ f64      │
╞════════╪══════════╡
│ 4      ┆ 1.84705  │
│ 5      ┆ 3.438095 │
│ 6      ┆ 2.613991 │
│ 7      ┆ 5.099062 │
│ 8      ┆ 12.43    │
│ …      ┆ …        │
│ 18     ┆ 3.274    │
│ 19     ┆ 2.620052 │
│ 20     ┆ 3.43361  │
│ 21     ┆ 2.99     │
│ 22     ┆ 2.09     │
└────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x10b15ebd0>, _boxhead=Boxhead([ColInfo(var='sku_id', type=<ColInfoTypeEnum.stub: 2>, column_label='sku_id', column_align='center', column_width=None), ColInfo(var='price', type=<ColInfoTypeEnum.default: 1>, column_label='Average Price ($)', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10daab9b0>, _spanners=Spanners([]), _heading=Heading(title='Alpha - Average SKU Retail Prices', subtitle=None, preheader=None), _stubhead='SKU IDs', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='sku_id', rows=[0]), grpname=None, colname='sku_id', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[1]), grpname=None, colname='sku_id', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[2]), grpname=None, colname='sku_id', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[3]), grpname=None, colname='sku_id', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[4]), grpname=None, colname='sku_id', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[5]), grpname=None, colname='sku_id', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[6]), grpname=None, colname='sku_id', rownum=6, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[7]), grpname=None, colname='sku_id', rownum=7, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[8]), grpname=None, colname='sku_id', rownum=8, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[9]), grpname=None, colname='sku_id', rownum=9, colnum=None, styles=[CellStyleText(color='#000000', font=None,

In [25]:
avg_sku_price.drop(pl.col('sku_id')).describe()

statistic,price
str,f64
"""count""",18.0
"""null_count""",0.0
"""mean""",4.042794
"""std""",2.947076
"""min""",1.84705
"""25%""",2.582056
"""50%""",2.99
"""75%""",3.438095
"""max""",12.43


#### Garbage Collect

In [26]:
exceptions = ['grocery_lf', 'sku_lf', 'kiwi_lf', 'In', 'exceptions', 'active_variables']

active_variables = [
    var for var, value in globals().items()
    if not var.startswith('_')   # Exclude variables that start with "_"
    and var not in exceptions    # Exclude variables in the exceptions list
    and isinstance(value, (pl.LazyFrame, pl.DataFrame, pl.Series, alt.Chart, alt.LayerChart, list, int, float, str, np.ndarray, np.int64, np.float32))  # Remove these types only
]

for var in active_variables:
    del globals()[var]
del active_variables, exceptions, var

gc.collect()

423

### Exploring Variation in Buyer Behaviour

#### Panellist-Level Data Preparation

<div style="max-width:600px;margin-left: auto; margin-right: auto;">
<img src="references\Consumer-Panel-Data-Relationship.png" width="600"/>
</div>

In [27]:
def trans_summary(brand, lf, year):
    
    # Primary Step: Filter by Year 1 and Remove Unused Columns
    filtered_lf = lf.filter(
        (pl.col('week') <= (year * 52)) &
        (pl.col('week') >= ((year - 1) * 52))
    ).drop(
        pl.col('week','sku_id')
    )

    # Intermediate Step: Group by trans_id, panel_id, and brand
    group_trans = filtered_lf.drop(
        pl.col('price', 'units')
    ).group_by(
        'trans_id', 'panel_id', 'brand'
    ).n_unique()
    
    if brand == "Category":
        # Panellist-level category transaction summary
        summary = group_trans.group_by(
            'panel_id'
        ).n_unique()
    else:
        # Panellist-level brand transaction summary
        summary = group_trans.filter(
            pl.col('brand') == brand
        ).group_by(
            'panel_id'
        ).n_unique()
    
    return summary.select(
        pl.col('panel_id'),
        pl.col('trans_id').alias('# of Purchases'),
        pl.col('brand').alias('Brands Purchased')
    )

In [28]:
def trans_pivot(lf, year):
    
    # Primary Step: Filter by Year 1 and Remove Unused Columns
    filtered_lf = lf.filter(
        (pl.col('week') <= (year * 52)) &
        (pl.col('week') >= ((year - 1) * 52))
    ).drop(
        pl.col('week','sku_id')
    )

    # Intermediate Step: Group by trans_id, panel_id, and brand
    group_trans = filtered_lf.drop(
        pl.col('price', 'units')
    ).group_by(
        'trans_id', 'panel_id', 'brand'
    ).n_unique()
    
    summary = group_trans.collect().pivot(
        on='brand',
        index='panel_id',
        values='panel_id',
        aggregate_function="len"
        
    ).join(
        other=group_trans.group_by('panel_id').n_unique().drop('brand').collect(),
        on='panel_id'
    ).rename(
        {'trans_id': 'Category'}
    ).drop(
        pl.col('panel_id')
    )
    
    return summary

In [29]:
def spend_summary(brand, lf, year):
    
    group_spend = lf.filter(
        (pl.col('week') <= (year * 52)) &
        (pl.col('week') >= ((year - 1) * 52))
    ).drop(
        pl.col('week','sku_id')
    ).with_columns(
        ((pl.col('units') * pl.col('price'))).alias('spend')
    )
    
    if brand == "Category":
        # Panellist-level category spend summary
        summary = group_spend.drop(
            pl.col('units', 'price', 'brand')
        ).group_by(
            'panel_id'
        ).agg(
            pl.col('spend').sum()
        )
    else:
        # Panellist-level brand spend summary
        summary = group_spend.drop(
            pl.col('units', 'price')
        ).group_by(
            'panel_id', 'brand'
        ).agg(
            pl.col('spend').sum()
        ).filter(
            pl.col('brand') == brand
        ).drop('brand')
        
    return summary

In [30]:
def vol_summary(brand, lf, year):
    
    group_vol = lf.filter(
        (pl.col('week') <= (year * 52)) &
        (pl.col('week') >= ((year - 1) * 52))
    ).join(
        other=sku_lf,
        left_on='sku_id',
        right_on='sku_id'
    ).drop(
        pl.col('week','sku_id')
    ).with_columns(
        # volume column that is the product of weight of each SKU and the units of SKU sold
        (((pl.col('units') * pl.col('weight'))/1000)).alias('volume') # # weight from grams to kilograms
    ).drop(
        pl.col('units', 'price', 'weight')
    )
    
    if brand == "Category":
        # Panellist-level category volume sales summary
        summary = group_vol.drop(
            pl.col('brand')
        ).group_by(
            'panel_id'
        ).agg(
            pl.col('volume').sum()
        )
    else:
        # Panellist-level brand volume sales summary
        summary = group_vol.group_by(
            'panel_id', 'brand'
        ).agg(
            pl.col('volume').sum()
        ).filter(
            pl.col('brand') == brand
        ).drop('brand')
        
    return summary

#### Examining Purchase Frequency

In [31]:
buyers = trans_pivot(grocery_lf, 1).count().select(
    'Alpha', 'Bravo', 'Charlie', 'Delta', 'Other', 'Category'
).unpivot(
    variable_name='brand',
    value_name='buyers'
)

(
    GT(buyers, rowname_col='brand')
    .tab_header(title='Number of Buyers in Year 1')
    .tab_stubhead(label="Brands/Category")
    .fmt_integer()
    .data_color(
        rows=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'],
        domain=[100, 3_000],
        palette=['white', 'rebeccapurple']
    )  
    .cols_label(buyers='# Of Transactions')
)

GT(_tbl_data=shape: (6, 2)
┌──────────┬────────┐
│ brand    ┆ buyers │
│ ---      ┆ ---    │
│ str      ┆ u32    │
╞══════════╪════════╡
│ Alpha    ┆ 2624   │
│ Bravo    ┆ 2562   │
│ Charlie  ┆ 813    │
│ Delta    ┆ 380    │
│ Other    ┆ 176    │
│ Category ┆ 4574   │
└──────────┴────────┘, _body=<great_tables._gt_data.Body object at 0x10db2bc20>, _boxhead=Boxhead([ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='brand', column_align='left', column_width=None), ColInfo(var='buyers', type=<ColInfoTypeEnum.default: 1>, column_label='# Of Transactions', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x102e84380>, _spanners=Spanners([]), _heading=Heading(title='Number of Buyers in Year 1', subtitle=None, preheader=None), _stubhead='Brands/Category', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[0]), grpname=None, colname='buyers', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7a4da6')]), StyleInfo(locname=LocBody(columns='buyers', rows=[1]), grpname=None, colname='buyers', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7d52a8')]), StyleInfo(locname=LocBody(columns='buyers', rows=[2]), grpname=None, colname='buyers', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d9cde6')]), StyleInfo(locname=LocBody(columns='buyers', rows=[3]), grpname=None, colname='buyers', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f0ebf5')]), StyleInfo(locname=LocBody(columns='buyers', rows=[4]), grpname=None, colname='buyers', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=Non

In [32]:
transactions = trans_pivot(grocery_lf, 1).sum().select(
    'Alpha', 'Bravo', 'Charlie', 'Delta', 'Other', 'Category'
).unpivot(
    variable_name='brand',
    value_name='transactions'
)

(
    GT(transactions, rowname_col='brand')
    .tab_header(title='Purchase Occasions by Buyers')
    .tab_stubhead(label="Brands/Category")
    .fmt_integer()
    .data_color(
        rows=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'],
        domain=[400, 10_000],
        palette=[ 'white', 'rebeccapurple']
    )  
    .cols_label(transactions='# Of Transactions')
)

GT(_tbl_data=shape: (6, 2)
┌──────────┬──────────────┐
│ brand    ┆ transactions │
│ ---      ┆ ---          │
│ str      ┆ u32          │
╞══════════╪══════════════╡
│ Alpha    ┆ 9060         │
│ Bravo    ┆ 8255         │
│ Charlie  ┆ 1882         │
│ Delta    ┆ 859          │
│ Other    ┆ 422          │
│ Category ┆ 20030        │
└──────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x10b15b6e0>, _boxhead=Boxhead([ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='brand', column_align='left', column_width=None), ColInfo(var='transactions', type=<ColInfoTypeEnum.default: 1>, column_label='# Of Transactions', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10da96690>, _spanners=Spanners([]), _heading=Heading(title='Purchase Occasions by Buyers', subtitle=None, preheader=None), _stubhead='Brands/Category', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[0]), grpname=None, colname='transactions', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7547a3')]), StyleInfo(locname=LocBody(columns='transactions', rows=[1]), grpname=None, colname='transactions', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8258ac')]), StyleInfo(locname=LocBody(columns='transactions', rows=[2]), grpname=None, colname='transactions', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e7e0ef')]), StyleInfo(locname=LocBody(columns='transactions', rows=[3]), grpname=None, colname='transactions', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f8f5fa')]), StyleInfo(locname=LocBody(columns='transactions', rows=[4]), grpname=None, colname='transactions', rownum=4, colnum=None, styles=[CellStyleText(color='#0

In [33]:
panel_size = 5021

penetration = buyers.with_columns(
    (pl.col('buyers') / panel_size).alias('penetration')
)

ppb = transactions.with_columns(
    (pl.col('transactions') / penetration.get_column('buyers')).alias('ppb')
)

In [34]:
(
    GT(penetration, rowname_col='brand')
    .tab_header(title='Penetration by Category & Brand')
    .tab_stubhead(label="Brands/Category")
    .fmt_percent()
    .data_color(
        rows=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'],
        domain=[0, 1],
        palette=['white', 'rebeccapurple']
    )  
    .cols_label(penetration='Penetration (%)')
    .cols_hide('buyers')
)

GT(_tbl_data=shape: (6, 3)
┌──────────┬────────┬─────────────┐
│ brand    ┆ buyers ┆ penetration │
│ ---      ┆ ---    ┆ ---         │
│ str      ┆ u32    ┆ f64         │
╞══════════╪════════╪═════════════╡
│ Alpha    ┆ 2624   ┆ 0.522605    │
│ Bravo    ┆ 2562   ┆ 0.510257    │
│ Charlie  ┆ 813    ┆ 0.16192     │
│ Delta    ┆ 380    ┆ 0.075682    │
│ Other    ┆ 176    ┆ 0.035053    │
│ Category ┆ 4574   ┆ 0.910974    │
└──────────┴────────┴─────────────┘, _body=<great_tables._gt_data.Body object at 0x10e44a3c0>, _boxhead=Boxhead([ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='brand', column_align='left', column_width=None), ColInfo(var='buyers', type=<ColInfoTypeEnum.hidden: 4>, column_label='buyers', column_align='center', column_width=None), ColInfo(var='penetration', type=<ColInfoTypeEnum.default: 1>, column_label='Penetration (%)', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x102e62050>, _spanners=Spanners([]), _heading=Heading(title='Penetration by Category & Brand', subtitle=None, preheader=None), _stubhead='Brands/Category', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[0]), grpname=None, colname='buyers', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[1]), grpname=None, colname='buyers', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[2]), grpname=None, colname='buyers', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[3]), grpname=None, colname='buyers', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(colo

In [35]:
(
    GT(ppb, rowname_col='brand')
    .tab_header(title='Purchases per Buyer (PPB)')
    .tab_stubhead(label="Brands/Category")
    .fmt_number()
    .data_color(
        rows=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'],
        domain=[0, 5],
        palette=['white', 'rebeccapurple']
    )  
    .cols_label(ppb='PPB')
    .cols_hide('transactions')
)

GT(_tbl_data=shape: (6, 3)
┌──────────┬──────────────┬──────────┐
│ brand    ┆ transactions ┆ ppb      │
│ ---      ┆ ---          ┆ ---      │
│ str      ┆ u32          ┆ f64      │
╞══════════╪══════════════╪══════════╡
│ Alpha    ┆ 9060         ┆ 3.452744 │
│ Bravo    ┆ 8255         ┆ 3.222092 │
│ Charlie  ┆ 1882         ┆ 2.314883 │
│ Delta    ┆ 859          ┆ 2.260526 │
│ Other    ┆ 422          ┆ 2.397727 │
│ Category ┆ 20030        ┆ 4.379099 │
└──────────┴──────────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x10e46f950>, _boxhead=Boxhead([ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='brand', column_align='left', column_width=None), ColInfo(var='transactions', type=<ColInfoTypeEnum.hidden: 4>, column_label='transactions', column_align='center', column_width=None), ColInfo(var='ppb', type=<ColInfoTypeEnum.default: 1>, column_label='PPB', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10e45c550>, _spanners=Spanners([]), _heading=Heading(title='Purchases per Buyer (PPB)', subtitle=None, preheader=None), _stubhead='Brands/Category', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[0]), grpname=None, colname='transactions', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[1]), grpname=None, colname='transactions', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[2]), grpname=None, colname='transactions', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[3]), grpname=None, colname='transactions', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=Non

#### Distribution of Purchase

##### Distribution of Category Purchase Frequency

In [36]:
category_trans_dist = (
    trans_summary('Category', grocery_lf, 1)
    .group_by(pl.col('# of Purchases').cast(pl.Int16))
    .agg(pl.len().cast(pl.Int32).alias("Frequency"))
    .collect() 
    .pipe(
        lambda df: df.vstack(
            pl.DataFrame(
                {
                    "# of Purchases": [0],
                    "Frequency": [panel_size - df["Frequency"].sum()],
                }
            ).with_columns(
                pl.col("# of Purchases").cast(pl.Int16),
                pl.col('Frequency').cast(pl.Int32)
                )
        )
    )
    .sort(by='# of Purchases')
    .with_columns(
        (pl.col('Frequency') / pl.col('Frequency').sum()).alias("% of Total")
    )    
)

(
    GT(category_trans_dist, rowname_col='# of Purchases')
    .tab_header(title='Category Purchase Distribution')
    .tab_stubhead(label="# of Purchases")
    .fmt_integer(columns='Frequency')
    .fmt_percent(columns='% of Total')
    .data_color(
        columns=['% of Total'],
        domain=[0, 0.2],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (24, 3)
┌────────────────┬───────────┬────────────┐
│ # of Purchases ┆ Frequency ┆ % of Total │
│ ---            ┆ ---       ┆ ---        │
│ i16            ┆ i32       ┆ f64        │
╞════════════════╪═══════════╪════════════╡
│ 0              ┆ 447       ┆ 0.089026   │
│ 1              ┆ 667       ┆ 0.132842   │
│ 2              ┆ 759       ┆ 0.151165   │
│ 3              ┆ 757       ┆ 0.150767   │
│ 4              ┆ 612       ┆ 0.121888   │
│ …              ┆ …         ┆ …          │
│ 19             ┆ 2         ┆ 0.000398   │
│ 20             ┆ 4         ┆ 0.000797   │
│ 22             ┆ 1         ┆ 0.000199   │
│ 25             ┆ 1         ┆ 0.000199   │
│ 27             ┆ 1         ┆ 0.000199   │
└────────────────┴───────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x10daf8f00>, _boxhead=Boxhead([ColInfo(var='# of Purchases', type=<ColInfoTypeEnum.stub: 2>, column_label='# of Purchases', column_align='right', column_width=None), ColInfo(var='Frequency', type=<ColInfoTypeEnum.default: 1>, column_label='Frequency', column_align='right', column_width=None), ColInfo(var='% of Total', type=<ColInfoTypeEnum.default: 1>, column_label='% of Total', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x102e486e0>, _spanners=Spanners([]), _heading=Heading(title='Category Purchase Distribution', subtitle=None, preheader=None), _stubhead='# of Purchases', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='% of Total', rows=[0]), grpname=None, colname='% of Total', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#bba4d2')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[1]), grpname=None, colname='% of Total', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9978bb')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[2]), grpname=None, colname='% of Total', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8b65b2')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[3]), grpname=None, colname='% of Total', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8c65b2')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[4]), grpname=None, colname='% of Total', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#a283c1')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[5]), grpname=None, colname='% of Total', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#b197cb')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[6]), grpname=None, colname='% of Total', rownum=6, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#c5b2d8')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[7]), grpname=None, colname='% of Total', rownum=7, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weigh

In [37]:
labels = [str(i) if i < 15 else '15+' for i in range(0, 16, 1)]
chart = freq_dist_plot(
    data=category_trans_dist,
    column="# of Purchases",
    bin_edges=np.arange(0, 15, 1),
    labels=labels,
    x_title="Number of Category Transactions",
    y_title="% of Households",
    chart_title="Category Transaction Distribution (%)",
    subtitle="Distribution of category purchasing in year 1",
    left_closed=False,
    compute_rel_freq=False
)
chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


##### Distribution of Brand Purchase Frequency

In [38]:
alpha_trans_dist = (
    trans_summary('Alpha', grocery_lf, 1)
    .group_by(pl.col('# of Purchases').cast(pl.Int16))
    .agg(pl.len().cast(pl.Int32).alias("Frequency"))
    .collect() 
    .pipe(
        lambda df: df.vstack(
            pl.DataFrame(
                {
                    "# of Purchases": [0],
                    "Frequency": [panel_size - df["Frequency"].sum()],
                }
            ).with_columns(
                pl.col("# of Purchases").cast(pl.Int16),
                pl.col('Frequency').cast(pl.Int32)
                )
        )
    )
    .sort(by='# of Purchases')
    .with_columns(
        (pl.col('Frequency') / pl.col('Frequency').sum()).alias("% of Total")
    )    
)

(
    GT(alpha_trans_dist, rowname_col='# of Purchases')
    .tab_header(title='Alpha Purchase Distribution')
    .tab_stubhead(label="# of Purchases")
    .fmt_integer(columns='Frequency')
    .fmt_percent(columns='% of Total')
    .data_color(
        columns=['% of Total'],
        domain=[0, 0.5],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (19, 3)
┌────────────────┬───────────┬────────────┐
│ # of Purchases ┆ Frequency ┆ % of Total │
│ ---            ┆ ---       ┆ ---        │
│ i16            ┆ i32       ┆ f64        │
╞════════════════╪═══════════╪════════════╡
│ 0              ┆ 2397      ┆ 0.477395   │
│ 1              ┆ 733       ┆ 0.145987   │
│ 2              ┆ 517       ┆ 0.102968   │
│ 3              ┆ 400       ┆ 0.079665   │
│ 4              ┆ 277       ┆ 0.055168   │
│ …              ┆ …         ┆ …          │
│ 14             ┆ 12        ┆ 0.00239    │
│ 15             ┆ 6         ┆ 0.001195   │
│ 16             ┆ 3         ┆ 0.000597   │
│ 18             ┆ 1         ┆ 0.000199   │
│ 20             ┆ 1         ┆ 0.000199   │
└────────────────┴───────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x10dad3160>, _boxhead=Boxhead([ColInfo(var='# of Purchases', type=<ColInfoTypeEnum.stub: 2>, column_label='# of Purchases', column_align='right', column_width=None), ColInfo(var='Frequency', type=<ColInfoTypeEnum.default: 1>, column_label='Frequency', column_align='right', column_width=None), ColInfo(var='% of Total', type=<ColInfoTypeEnum.default: 1>, column_label='% of Total', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10b326990>, _spanners=Spanners([]), _heading=Heading(title='Alpha Purchase Distribution', subtitle=None, preheader=None), _stubhead='# of Purchases', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='% of Total', rows=[0]), grpname=None, colname='% of Total', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#6d3c9e')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[1]), grpname=None, colname='% of Total', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d2c3e1')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[2]), grpname=None, colname='% of Total', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#dfd5ea')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[3]), grpname=None, colname='% of Total', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e7deef')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[4]), grpname=None, colname='% of Total', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#eee8f4')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[5]), grpname=None, colname='% of Total', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f1edf6')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[6]), grpname=None, colname='% of Total', rownum=6, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f6f3f9')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[7]), grpname=None, colname='% of Total', rownum=7, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=N

In [39]:
labels = [str(i) if i < 10 else '10+' for i in range(11)]

chart = freq_dist_plot(
    data=alpha_trans_dist,
    column="# of Purchases",
    bin_edges=np.arange(0, 10, 1),
    labels=labels,
    x_title="Number of Transactions",
    y_title="% of Households",
    chart_title="Alpha Transaction Distribution (%)",
    subtitle="Distribution of purchase frequency for Alpha in year 1",
    left_closed=False,
    compute_rel_freq=False
)
chart.show()


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


#### Distribution of Spend

##### Distribution of Category Spend

In [40]:
category_spend = spend_summary('Category', grocery_lf, 1).drop('panel_id')

category_spend.describe()

statistic,spend
str,f64
"""count""",4574.0
"""null_count""",0.0
"""mean""",15.763424
"""std""",12.876024
"""min""",1.27
"""25%""",6.76
"""50%""",12.57
"""75%""",20.74
"""max""",166.699936


In [41]:
percentiles = np.arange(5, 96, 5)

percentile_values = np.percentile(category_spend.select('spend').collect(), percentiles)

percentile_df = pl.DataFrame({'Percentile Level': percentiles,
                              'Percentile': percentile_values})

(
    GT(percentile_df, rowname_col='Percentile Level')
    .tab_header(title='Category Spend Percentile')
    .tab_stubhead(label="Percentile Level")
    .fmt_number()
    .data_color(
        domain=[0, 40],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (19, 2)
┌──────────────────┬────────────┐
│ Percentile Level ┆ Percentile │
│ ---              ┆ ---        │
│ i64              ┆ f64        │
╞══════════════════╪════════════╡
│ 5                ┆ 2.69       │
│ 10               ┆ 3.39       │
│ 15               ┆ 4.67       │
│ 20               ┆ 5.88       │
│ 25               ┆ 6.76       │
│ …                ┆ …          │
│ 75               ┆ 20.7375    │
│ 80               ┆ 23.43      │
│ 85               ┆ 26.880501  │
│ 90               ┆ 31.809999  │
│ 95               ┆ 39.718504  │
└──────────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x10e461220>, _boxhead=Boxhead([ColInfo(var='Percentile Level', type=<ColInfoTypeEnum.stub: 2>, column_label='Percentile Level', column_align='right', column_width=None), ColInfo(var='Percentile', type=<ColInfoTypeEnum.default: 1>, column_label='Percentile', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10db0fbd0>, _spanners=Spanners([]), _heading=Heading(title='Category Spend Percentile', subtitle=None, preheader=None), _stubhead='Percentile Level', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Percentile Level', rows=[0]), grpname=None, colname='Percentile Level', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ece6f2')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[1]), grpname=None, colname='Percentile Level', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d9cce6')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[2]), grpname=None, colname='Percentile Level', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#c6b2d9')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[3]), grpname=None, colname='Percentile Level', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#b299cc')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[4]), grpname=None, colname='Percentile Level', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9f80bf')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[5]), grpname=None, colname='Percentile Level', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8c66b2')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[6]), grpname=None, colname='Percentile Level', rownum=6, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#794ca6')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[7]), grpname=None, colname='Percentile Level', rownum=7, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[8]), grpname=None, colname='Percenti

In [42]:
labels = [f"{lb} - {lb + 5 if lb != 50 else '∞'}" for lb in range(-5, 55, 5)]
chart = freq_dist_plot(
    data=category_spend,
    column="spend",
    bin_edges=np.arange(0, 51, 5),
    labels=labels,
    x_title="Category Spend ($)",
    y_title="% of Households",
    chart_title="Category Spend Distribution (%)",
    subtitle="Distribution of category spend across category buyers"
)
chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


##### Distribution of Brand Spend

In [43]:
alpha_spend = spend_summary('Alpha', grocery_lf, 1).drop('panel_id')

alpha_spend.describe()

statistic,spend
str,f64
"""count""",2624.0
"""null_count""",0.0
"""mean""",12.793803
"""std""",11.715616
"""min""",1.57
"""25%""",3.69
"""50%""",9.67
"""75%""",17.130001
"""max""",105.899971


In [44]:
percentiles = np.arange(5, 96, 5)

percentile_values = np.percentile(alpha_spend.select('spend').collect(), percentiles)

percentile_df = pl.DataFrame({'Percentile Level': percentiles,
                              'Percentile': percentile_values})

(
    GT(percentile_df, rowname_col='Percentile Level')
    .tab_header(title='Alpha Spend Percentile')
    .tab_stubhead(label="Percentile Level")
    .fmt_number()
    .data_color(
        domain=[0, 40],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (19, 2)
┌──────────────────┬────────────┐
│ Percentile Level ┆ Percentile │
│ ---              ┆ ---        │
│ i64              ┆ f64        │
╞══════════════════╪════════════╡
│ 5                ┆ 2.453      │
│ 10               ┆ 2.79       │
│ 15               ┆ 3.38       │
│ 20               ┆ 3.49       │
│ 25               ┆ 3.69       │
│ …                ┆ …          │
│ 75               ┆ 17.130001  │
│ 80               ┆ 20.040001  │
│ 85               ┆ 23.1355    │
│ 90               ┆ 27.62      │
│ 95               ┆ 35.485003  │
└──────────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x10e462300>, _boxhead=Boxhead([ColInfo(var='Percentile Level', type=<ColInfoTypeEnum.stub: 2>, column_label='Percentile Level', column_align='right', column_width=None), ColInfo(var='Percentile', type=<ColInfoTypeEnum.default: 1>, column_label='Percentile', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10e4642f0>, _spanners=Spanners([]), _heading=Heading(title='Alpha Spend Percentile', subtitle=None, preheader=None), _stubhead='Percentile Level', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Percentile Level', rows=[0]), grpname=None, colname='Percentile Level', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ece6f2')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[1]), grpname=None, colname='Percentile Level', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d9cce6')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[2]), grpname=None, colname='Percentile Level', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#c6b2d9')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[3]), grpname=None, colname='Percentile Level', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#b299cc')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[4]), grpname=None, colname='Percentile Level', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9f80bf')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[5]), grpname=None, colname='Percentile Level', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8c66b2')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[6]), grpname=None, colname='Percentile Level', rownum=6, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#794ca6')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[7]), grpname=None, colname='Percentile Level', rownum=7, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[8]), grpname=None, colname='Percentile 

In [45]:
labels = [f"{lb} - {lb + 2 if lb != 40 else '∞'}" for lb in range(-2, 41, 2)]
chart = freq_dist_plot(
    data=alpha_spend,
    column="spend",
    bin_edges=np.arange(0, 41, 2),
    labels=labels,
    x_title="Brand Spend ($)",
    y_title="% of Households",
    chart_title="Alpha Spend Distribution (%)",
    subtitle="Distribution of spend on Alpha across brand buyers",
    label_angle=-45
)
chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


##### Analyzing Alpha's One-Time Buyers in Year 1

In [46]:
# One-Time Alpha Buyers
one_time_alpha_buyers = trans_summary('Alpha',grocery_lf,1).filter(pl.col('# of Purchases') == 1).collect()
one_time_alpha_buyers

# Alpha buyers who spent between $2 to $4
spend_range_alpha_buyers = spend_summary('Alpha', grocery_lf, 1).filter(
    (pl.col('spend') >= 2) &
    (pl.col('spend') < 4)
).collect()

res = one_time_alpha_buyers.with_columns(
    contains=pl.col('panel_id').is_in(spend_range_alpha_buyers.select('panel_id'))
).select('contains').sum().item()

print(f'There were {one_time_alpha_buyers.count().item(0,'panel_id')} one-time buyers of Alpha brand in Year 1.')
print(f'{res} of those one-time Alpha buyers spent between [$2, $4).')
print(f'Given that there are {spend_range_alpha_buyers.count().item(0,'spend')} Alpha buyers in total who spent between $2-$4,')
print(f'the {res} one-time buyers represent {res / spend_range_alpha_buyers.count().item(0,'spend'):.0%} of the Alpha buyers in that spend range.')

There were 733 one-time buyers of Alpha brand in Year 1.
623 of those one-time Alpha buyers spent between [$2, $4).
Given that there are 633 Alpha buyers in total who spent between $2-$4,
the 623 one-time buyers represent 98% of the Alpha buyers in that spend range.


**Validate**:

In [47]:
spend_range_alpha_buyers.filter(
    pl.col('panel_id').is_in(one_time_alpha_buyers.select('panel_id'))
).select(pl.col('spend').sum()).item()

1937.2301025390625

In [48]:
grocery_lf.filter(
    (pl.col('week') <= 52) &
    (pl.col('panel_id').is_in(spend_range_alpha_buyers.select('panel_id'))) &  # Assuming panel_id is the relevant column
    (pl.col('panel_id').is_in(one_time_alpha_buyers.select('panel_id'))) &  # Same assumption
    (pl.col('brand') == 'Alpha')
).with_columns(
    ((pl.col('units') * pl.col('price'))).alias('spend')
).select(pl.col('spend').sum()).collect().item()

1937.229736328125

#### Distribution of Volume Purchase

##### Distribution of Category Volume Purchase Frequency

In [49]:
category_vol = vol_summary('Category', grocery_lf, 1).drop('panel_id')

category_vol.describe()

statistic,volume
str,f64
"""count""",4574.0
"""null_count""",0.0
"""mean""",4.544163
"""std""",3.884697
"""min""",0.25
"""25%""",2.0
"""50%""",3.5
"""75%""",6.0
"""max""",67.0


In [50]:
percentiles = np.arange(5, 96, 5)

percentile_values = np.percentile(category_vol.select('volume').collect(), percentiles)

percentile_df = pl.DataFrame({'Percentile Level': percentiles,
                              'Percentile': percentile_values})

(
    GT(percentile_df, rowname_col='Percentile Level')
    .tab_header(title='Category Volume Percentile')
    .tab_stubhead(label="Percentile Level")
    .fmt_number()
    .data_color(
        domain=[0, 15],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (19, 2)
┌──────────────────┬────────────┐
│ Percentile Level ┆ Percentile │
│ ---              ┆ ---        │
│ i64              ┆ f64        │
╞══════════════════╪════════════╡
│ 5                ┆ 0.8325     │
│ 10               ┆ 1.0        │
│ 15               ┆ 1.0        │
│ 20               ┆ 1.5        │
│ 25               ┆ 2.0        │
│ …                ┆ …          │
│ 75               ┆ 6.0        │
│ 80               ┆ 7.0        │
│ 85               ┆ 8.0        │
│ 90               ┆ 9.0        │
│ 95               ┆ 12.0       │
└──────────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x10e46bca0>, _boxhead=Boxhead([ColInfo(var='Percentile Level', type=<ColInfoTypeEnum.stub: 2>, column_label='Percentile Level', column_align='right', column_width=None), ColInfo(var='Percentile', type=<ColInfoTypeEnum.default: 1>, column_label='Percentile', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10da672b0>, _spanners=Spanners([]), _heading=Heading(title='Category Volume Percentile', subtitle=None, preheader=None), _stubhead='Percentile Level', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Percentile Level', rows=[0]), grpname=None, colname='Percentile Level', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ccbbdd')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[1]), grpname=None, colname='Percentile Level', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9977bb')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[2]), grpname=None, colname='Percentile Level', rownum=2, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[3]), grpname=None, colname='Percentile Level', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[4]), grpname=None, colname='Percentile Level', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[5]), grpname=None, colname='Percentile Level', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[6]), grpname=None, colname='Percentile Level', rownum=6, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[7]), grpname=None, colname='Percentile Level', rownum=7, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[8]), grpname=None, colname='Percent

In [51]:
labels = [f"{lb} - {lb + 1 if lb != 12 else '∞'}" for lb in range(-1, 13, 1)]
chart = freq_dist_plot(
    data=category_vol,
    column="volume",
    bin_edges=np.arange(0, 13, 1),
    labels=labels,
    x_title="Category Volume Purchase ($)",
    y_title="% of Households",
    chart_title="Category Volume Purchase Distribution (%)",
    subtitle="Distribution of category volume purchases across category buyers"
)
chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


##### Distribution of Brand Volume Purchase Frequency

In [52]:
alpha_vol = vol_summary('Alpha', grocery_lf, 1).drop('panel_id')

alpha_vol.describe()

statistic,volume
str,f64
"""count""",2624.0
"""null_count""",0.0
"""mean""",3.493236
"""std""",3.350901
"""min""",0.25
"""25%""",1.0
"""50%""",2.5
"""75%""",5.0
"""max""",27.55


In [53]:
percentiles = np.arange(5, 96, 5)

percentile_values = np.percentile(alpha_vol.select('volume').collect(), percentiles)

percentile_df = pl.DataFrame({'Percentile Level': percentiles,
                              'Percentile': percentile_values})

(
    GT(percentile_df, rowname_col='Percentile Level')
    .tab_header(title='Alpha Volume Percentile')
    .tab_stubhead(label="Percentile Level")
    .fmt_number()
    .data_color(
        domain=[0, 15],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (19, 2)
┌──────────────────┬────────────┐
│ Percentile Level ┆ Percentile │
│ ---              ┆ ---        │
│ i64              ┆ f64        │
╞══════════════════╪════════════╡
│ 5                ┆ 0.5        │
│ 10               ┆ 0.5        │
│ 15               ┆ 1.0        │
│ 20               ┆ 1.0        │
│ 25               ┆ 1.0        │
│ …                ┆ …          │
│ 75               ┆ 5.0        │
│ 80               ┆ 5.5        │
│ 85               ┆ 6.35       │
│ 90               ┆ 8.0        │
│ 95               ┆ 10.0       │
└──────────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x10e42aee0>, _boxhead=Boxhead([ColInfo(var='Percentile Level', type=<ColInfoTypeEnum.stub: 2>, column_label='Percentile Level', column_align='right', column_width=None), ColInfo(var='Percentile', type=<ColInfoTypeEnum.default: 1>, column_label='Percentile', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10db30e90>, _spanners=Spanners([]), _heading=Heading(title='Alpha Volume Percentile', subtitle=None, preheader=None), _stubhead='Percentile Level', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Percentile Level', rows=[0]), grpname=None, colname='Percentile Level', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ccbbdd')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[1]), grpname=None, colname='Percentile Level', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9977bb')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[2]), grpname=None, colname='Percentile Level', rownum=2, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[3]), grpname=None, colname='Percentile Level', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[4]), grpname=None, colname='Percentile Level', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[5]), grpname=None, colname='Percentile Level', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[6]), grpname=None, colname='Percentile Level', rownum=6, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[7]), grpname=None, colname='Percentile Level', rownum=7, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[8]), grpname=None, colname='Percentile

In [54]:
labels = [f"{lb} - {lb + 1 if lb != 9 else '∞'}" for lb in range(-1, 10, 1)]

chart = freq_dist_plot(
    data=alpha_vol,
    column="volume",
    bin_edges=np.arange(0, 10, 1),
    labels=labels,
    x_title="Brand Volume Purchase ($)",
    y_title="% of Households",
    chart_title="Alpha Brand Volume Purchase Distribution (%)",
    subtitle="Distribution of Alpha volume purchases across brand buyers"
)
chart.show()

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


##### Garbage Collect

In [55]:
exceptions = ['grocery_lf', 'sku_lf', 'kiwi_lf', 'In', 'exceptions', 'active_variables',
              'penetration', 'ppb', 'panel_size']

active_variables = [
    var for var, value in globals().items()
    if not var.startswith('_')   # Exclude variables that start with "_"
    and var not in exceptions    # Exclude variables in the exceptions list
    and isinstance(value, (pl.LazyFrame, pl.DataFrame, pl.Series, alt.Chart, alt.LayerChart, list, int, float, str, np.ndarray, np.int64, np.float32))  # Remove these types only
]

for var in active_variables:
    del globals()[var]
del active_variables, exceptions, var

gc.collect()

207

#### Performing a Basic Decile Analysis

##### Decile analysis of category buying behaviour - Decile based on category buyers

In [56]:
# Part 1: Break-tie ranking of total category spend data
# Part 2: Decile cuts in ranking
buyer_decile = spend_summary(
    'Category', grocery_lf, 1
).with_columns(
    pl.col('spend').rank(method='ordinal', descending=True).alias('rank'),
).with_columns(
    (np.floor(10 * (pl.col('rank') - 1) / pl.col('rank').max()) + 1).cast(pl.UInt16).alias('decile')
).sort('decile', 'rank').collect()

buyer_decile

panel_id,spend,rank,decile
i32,f32,u32,u16
3111594,166.699936,1,1
3119170,108.829994,2,1
3125418,102.87001,3,1
3110132,100.630013,4,1
3109647,97.37999,5,1
…,…,…,…
3128086,1.49,4570,10
3112878,1.39,4571,10
3120986,1.33,4572,10


In [ ]:
# Helper functions
def calculate_percentage(series):
    return series / series.sum()

def group_aggregate(data, group_col, agg_col, agg_expr):
    return data.group_by(group_col).agg(agg_expr(pl.col(agg_col)))

# Decile table creation
decile_tab = buyer_decile.select("decile").unique().sort("decile").rename({"decile": "Decile"})

# Populate decile statistics
# Decile counts and % HH
decile_counts = group_aggregate(buyer_decile, 'decile', 'panel_id', pl.Expr.count)
decile_tab.insert_column(1, pl.lit(calculate_percentage(decile_counts['panel_id'])).alias('% HH')) 

# % Spend
decile_spend = group_aggregate(buyer_decile, 'decile', 'spend', pl.Expr.sum)
decile_tab.insert_column(2, pl.lit(calculate_percentage(decile_spend['spend'])).alias('% Spend')) 

# % Transactions
group_trans = trans_summary('Category', grocery_lf, 1).collect().join(
    other=buyer_decile.group_by('panel_id').agg(pl.col('decile').sum()),
    on='panel_id'
).sort('decile')
decile_trans = group_aggregate(group_trans, 'decile', '# of Purchases', pl.Expr.sum)
decile_tab.insert_column(3, pl.lit(calculate_percentage(decile_trans['# of Purchases'])).alias('% Trans')) 

# Spend/HH, Cat Trans/HH, Avg Spend/Trans
decile_tab.insert_column(4, pl.lit(decile_spend['spend'] / decile_counts['panel_id']).alias('Spend/HH'))
decile_tab.insert_column(5, pl.lit(decile_trans['# of Purchases'] / decile_counts['panel_id']).alias('Cat Trans/HH'))
decile_tab.insert_column(6, pl.lit(decile_spend['spend'] / decile_trans['# of Purchases']).alias('Avg Spend/Trans'))

# Unique Brands Purchased
unique_brands = group_aggregate(group_trans, 'decile', 'Brands Purchased', pl.Expr.mean)
decile_tab.insert_column(7, pl.lit(unique_brands['Brands Purchased']).alias('# Unique Brands'))

(
    GT(decile_tab, rowname_col='Decile')
    .tab_header(title='Decile Analysis of Category Buying Behavior',
                subtitle= 'Each decile equals 10% category buyers')
    .tab_stubhead(label="Decile")
    .fmt_percent(columns=['% HH', '% Spend', '% Trans'], decimals=1)
    .fmt_number(columns=['Cat Trans/HH', '# Unique Brands'])
    .fmt_currency(columns=['Spend/HH','Avg Spend/Trans'])
)

Decile,% HH,% Spend,% Trans,Spend/HH,Cat Trans/HH,Avg Spend/Trans,# Unique Brands
u16,f64,f32,f64,f64,f64,f64,f64
1,0.100131,0.283613,0.242786,44.648476,10.617904,4.205018,1.847162
2,0.099913,0.172067,0.163005,27.147378,7.14442,3.799801,1.636761
3,0.100131,0.133154,0.134548,20.962076,5.884279,3.562386,1.582969
4,0.099913,0.107161,0.111083,16.906979,4.868709,3.472579,1.540481
5,0.099913,0.08755,0.094708,13.812933,4.150985,3.327628,1.498906
6,0.100131,0.070741,0.077833,11.136638,3.40393,3.2717,1.412664
7,0.099913,0.056896,0.067099,8.976631,2.940919,3.052322,1.374179
8,0.100131,0.042816,0.048777,6.740415,2.133188,3.159785,1.224891
9,0.099913,0.028881,0.036895,4.556586,1.617068,2.817808,1.212254


##### Decile analysis of category buying behaviour - Decile based on category spend/revenue

In [61]:
spend_total = spend_summary('Category', grocery_lf, 1).select(pl.col('spend')).sum().collect().to_series()[0]
breaks = np.arange(0, spend_total, spend_total/10)
labels = [f'{i}' for i in range(0, 11, 1)]

spend_decile = spend_summary(
    'Category', grocery_lf, 1
).sort(
    'spend', descending=True
).with_columns(
    pl.col('spend').cum_sum().alias('cum sum')
).with_columns(
    pl.col('cum sum').cut(
        breaks=breaks,
        labels=labels
    ).cast(pl.UInt16).alias('decile')
).sort('decile').collect()

spend_decile

panel_id,spend,cum sum,decile
i32,f32,f32,u16
3111594,166.699936,166.699936,1
3119170,108.829994,275.529938,1
3125418,102.87001,378.399963,1
3110132,100.630013,479.029968,1
3109647,97.37999,576.409973,1
…,…,…,…
3128086,1.49,72096.984375,10
3112878,1.39,72098.375,10
3120986,1.33,72099.703125,10


In [62]:
# Helper functions
def calculate_percentage(series):
    return series / series.sum()

def group_aggregate(data, group_col, agg_col, agg_expr):
    return data.group_by(group_col).agg(agg_expr(pl.col(agg_col)))

# Decile table creation
decile_tab_spend = spend_decile.select("decile").unique().sort("decile").rename({"decile": "Decile"})

# Populate decile statistics
# Decile counts and % HH
decile_counts = group_aggregate(spend_decile, 'decile', 'panel_id', pl.Expr.count)
decile_tab_spend.insert_column(1, pl.lit(calculate_percentage(decile_counts['panel_id'])).alias('% HH')) 

# % Spend
decile_spend = group_aggregate(spend_decile, 'decile', 'spend', pl.Expr.sum)
decile_tab_spend.insert_column(2, pl.lit(calculate_percentage(decile_spend['spend'])).alias('% Spend')) 

# % Transactions
group_trans = trans_summary('Category', grocery_lf, 1).collect().join(
    other=spend_decile.group_by('panel_id').agg(pl.col('decile').sum()),
    on='panel_id'
).sort('decile')
decile_trans = group_aggregate(group_trans, 'decile', '# of Purchases', pl.Expr.sum)
decile_tab_spend.insert_column(3, pl.lit(calculate_percentage(decile_trans['# of Purchases'])).alias('% Trans')) 

# Spend/HH, Cat Trans/HH, Avg Spend/Trans
decile_tab_spend.insert_column(4, pl.lit(decile_spend['spend'] / decile_counts['panel_id']).alias('Spend/HH'))
decile_tab_spend.insert_column(5, pl.lit(decile_trans['# of Purchases'] / decile_counts['panel_id']).alias('Cat Trans/HH'))
decile_tab_spend.insert_column(6, pl.lit(decile_spend['spend'] / decile_trans['# of Purchases']).alias('Avg Spend/Trans'))

# # Unique Brands Purchased
unique_brands = group_aggregate(group_trans, 'decile', 'Brands Purchased', pl.Expr.mean)
decile_tab_spend.insert_column(7, pl.lit(unique_brands['Brands Purchased']).alias('# Unique Brands'))

(
    GT(decile_tab_spend, rowname_col='Decile')
    .tab_header(title='Decile Analysis of Category Buying Behavior',
                subtitle= 'Each decile equals 10% category buyers')
    .tab_stubhead(label="Decile")
    .fmt_percent(columns=['% HH', '% Spend', '% Trans'], decimals=1)
    .fmt_number(columns=['Cat Trans/HH', '# Unique Brands'])
    .fmt_currency(columns=['Spend/HH','Avg Spend/Trans'])
)

GT(_tbl_data=shape: (10, 8)
┌────────┬──────────┬──────────┬──────────┬───────────┬──────────────┬──────────────────┬──────────┐
│ Decile ┆ % HH     ┆ % Spend  ┆ % Trans  ┆ Spend/HH  ┆ Cat Trans/HH ┆ Avg Spend/Trans  ┆ # Unique │
│ ---    ┆ ---      ┆ ---      ┆ ---      ┆ ---       ┆ ---          ┆ ---              ┆ Brands   │
│ u16    ┆ f64      ┆ f32      ┆ f64      ┆ f64       ┆ f64          ┆ f64              ┆ ---      │
│        ┆          ┆          ┆          ┆           ┆              ┆                  ┆ f64      │
╞════════╪══════════╪══════════╪══════════╪═══════════╪══════════════╪══════════════════╪══════════╡
│ 1      ┆ 0.024705 ┆ 0.099946 ┆ 0.075836 ┆ 63.772496 ┆ 13.442478    ┆ 4.744103         ┆ 1.920354 │
│ 2      ┆ 0.037167 ┆ 0.099811 ┆ 0.089116 ┆ 42.332709 ┆ 10.5         ┆ 4.031687         ┆ 1.823529 │
│ 3      ┆ 0.046349 ┆ 0.099887 ┆ 0.092411 ┆ 33.971984 ┆ 8.731132     ┆ 3.890903         ┆ 1.783019 │
│ 4      ┆ 0.055969 ┆ 0.100177 ┆ 0.094159 ┆ 28.214609 ┆ 7.3671875    ┆ 3.829767         ┆ 1.628906 │
│ 5      ┆ 0.066463 ┆ 0.099893 ┆ 0.098552 ┆ 23.692237 ┆ 6.493421     ┆ 3.648652         ┆ 1.648026 │
│ 6      ┆ 0.078924 ┆ 0.100206 ┆ 0.102097 ┆ 20.013875 ┆ 5.66482      ┆ 3.533012         ┆ 1.573407 │
│ 7      ┆ 0.094666 ┆ 0.100073 ┆ 0.103744 ┆ 16.663905 ┆ 4.799076     ┆ 3.472315         ┆ 1.535797 │
│ 8      ┆ 0.116747 ┆ 0.099911 ┆ 0.108737 ┆ 13.490187 ┆ 4.078652     ┆ 3.307511         ┆ 1.477528 │
│ 9      ┆ 0.156756 ┆ 0.100032 ┆ 0.112282 ┆ 10.059274 ┆ 3.136681     ┆ 3.206981         ┆ 1.390516 │
│ 10     ┆ 0.322256 ┆ 0.100065 ┆ 0.123065 ┆ 4.894757  ┆ 1.67232      ┆ 2.926925         ┆ 1.162144 │
└────────┴──────────┴──────────┴──────────┴───────────┴──────────────┴──────────────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x10e4bb750>, _boxhead=Boxhead([ColInfo(var='Decile', type=<ColInfoTypeEnum.stub: 2>, column_label='Decile', column_align='center', column_width=None), ColInfo(var='% HH', type=<ColInfoTypeEnum.default: 1>, column_label='% HH', column_align='right', column_width=None), ColInfo(var='% Spend', type=<ColInfoTypeEnum.default: 1>, column_label='% Spend', column_align='right', column_width=None), ColInfo(var='% Trans', type=<ColInfoTypeEnum.default: 1>, column_label='% Trans', column_align='right', column_width=None), ColInfo(var='Spend/HH', type=<ColInfoTypeEnum.default: 1>, column_label='Spend/HH', column_align='right', column_width=None), ColInfo(var='Cat Trans/HH', type=<ColInfoTypeEnum.default: 1>, column_label='Cat Trans/HH', column_align='right', column_width=None), ColInfo(var='Avg Spend/Trans', type=<ColInfoTypeEnum.default: 1>, column_label='Avg Spend/Trans', column_align='right', column_width=None), ColInfo(var='# Unique Brands', type=<ColInfoTypeEnum.default: 1>, column_label='# Unique Brands', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x10db322d0>, _spanners=Spanners([]), _heading=Heading(title='Decile Analysis of Category Buying Behavior', subtitle='Each decile equals 10% category buyers', preheader=None), _stubhead='Decile', _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x10db32090>, _formats=[<great_tables._gt_data.FormatInfo object at 0x10db2bd80>, <great_tables._gt_data.FormatInfo object at 0x10e423d90>, <great_tables._gt_data.FormatInfo object at 0x10e420d70>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#F